In [4]:
import pandas as pd

In [2]:
# there are ratios in the data, can they be used?
# (I dont think interpretations alone are going to cut it)
# So what can I do with these ratios?

In [3]:
# ooh, if I can find something like LCR(ofc, excepting other ratios such as quick), then I can generate a 
# basic basel report> bool:compliant/not
# you dont really need uslocs for this cos you dont need large time data, you are calculating assets for a point
# so, use balance sheets

In [4]:
"""I can do simple metrics calculaitons and maybe predict them, but the regression does not seem to be working"""

'I can do simple metrics calculaitons and maybe predict them, but the regression does not seem to be working'

In [5]:
# quick ratio, cash ratio, net working capital ratio, curretn ratio vs liq ratio

#Tier 1 (core) risk-based capital
#Tier 2 risk-based capital
#Equity capital to assets
#Core capital (leverage) ratio

In [6]:
# commonstk = 3020000000
# toteqtycap = 236206000000
# riskwtd_assets = 1394763000000
# toteqtycap/riskwtd_assets * 100

## Basic Stats Analysis

In [5]:
df = pd.read_csv("../../data/raw/fin_statements/jpm_bs.csv")

In [6]:
df.head(10)
"""Last col -20 is all wrong, actually -grey cols"""

'Last col -20 is all wrong, actually -grey cols'

### Liquidity

In [45]:
ca = ['Cash, Cash Equivalents & Federal Funds Sold',
      'Trading Securities', 'Other Short Term Investments', 'Loans Held for Sale', 'Accounts receivable', 
       'Other Assets', 'Consumer Loan', 'Held To Maturity Securities']
#Consumer Loan , 'Held To Maturity Securities'


# cce: 'Cash And Cash Equivalents', 'Cash', 'Cash And Due from Banks','Interest Bearing Deposits Assets', 'Money Market Investments',

 # this includes cash, cash eq, marketable securities, accs recievable
# I am not really seeing prepaid expenses or inventory on the balance sheet - they all seem quick enough to me
# or shld I split qa along trading securities, other short term inv, loans held for sale
qa = ['Cash, Cash Equivalents & Federal Funds Sold',
      'Trading Securities', 'Accounts receivable']
#, 'Available for Sale Securities'


cl = ['Total Deposits','Federal Funds Purchased And Securities Sold Under Agreement To Repurchase', 'Payables And Accrued Expenses',
      'Current Debt And Capital Lease Obligation', 'Trading Liabilities']
# ambiguous = derviative assets, HMS, 


#### Quick ratio & Current Ratio

In [46]:
def get_qancl(df): # all rows of quick a and curr liab, ca for all banks
    quick_assets = []
    current_liabilities = []
    current_assets = []
    for idx, row in df.iterrows():
        if row['Tag'] in qa:
            quick_assets.append(row)
        if row["Tag"] in cl:
            current_liabilities.append(row)
        if row["Tag"] in ca:
            current_assets.append(row)
    return quick_assets, current_assets, current_liabilities
    
    # len(current_assets), len(current_liabilities)


In [59]:
def get_qancl_yr(yr, bank, quick_assets, current_assets, current_liabilities): # total canl for yr 
    # pass yr as index : '23-2, '22 - 3, '21-4, '20-5 
    qa_yr = 0
    cl_yr = 0
    ca_yr = 0
    for asset in quick_assets:
        if asset[1] == bank:
            # print(asset[yr])
            val = int(asset[yr].replace(',', "").replace("--", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', "")) 
                val = val/3
            qa_yr += val

    
    for lia in current_liabilities:
        if lia[1] == bank:
            val = int(lia[yr].replace(',', "").replace("--", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(lia[i].replace(',', "")) 
                val = val/3
            cl_yr += val

    for asset in current_assets:
        if asset[1] == bank:
            val = int(asset[yr].replace(',', "").replace("--", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', "")) 
                val = val/3
            ca_yr += val

    return qa_yr*1000, ca_yr*1000, cl_yr*1000  # for yr for bank
# all yf values are in thousands



In [63]:
quick_assets, current_assets, current_liabilities = get_qancl(df)
current_liabilities

qa_23, ca_23, cl_23 = get_qancl_yr(3, 'JPM', quick_assets, current_assets, current_liabilities)
cl_23, qa_23, ca_23
quick_ratio = qa_23/cl_23 
current_ratio = ca_23/cl_23

In [64]:
quick_ratio, qa_23/10**9, cl_23/10**9

(0.44803211646705765, 1368.127, 3053.636)

In [62]:
current_ratio, ca_23/10**9

(0.8262422783357429, 2577.438)

In [35]:
# the grey cols are wrong - rescrape and check incom, cash flow also :===

#### DataFrames

In [65]:
df.head()

,Tag,Bank,12/31/2023,12/31/2022,12/31/2021,12/31/2020
0,Total Assets,JPM,"3,875,393,000","3,665,743,000","3,743,567,000","3,384,757,000"
1,"Cash, Cash Equivalents & Federal Funds Sold",JPM,"900,303,000","882,826,000","1,002,532,000","823,893,000"
2,Cash And Cash Equivalents,JPM,"624,151,000","567,234,000","740,834,000","527,609,000"
3,Cash,JPM,"29,066,000","27,697,000","26,438,000","24,874,000"
4,Cash And Due from Banks,JPM,"595,085,000","539,537,000","714,396,000","502,735,000"


In [66]:
tdf = df.transpose()
tdf.columns = tdf.iloc[0]
tdf = tdf.iloc[1:, :]
tdf.head()

Tag,Total Assets,"Cash, Cash Equivalents & Federal Funds Sold",Cash And Cash Equivalents,Cash,Cash And Due from Banks,Restricted Cash And Investments,Restricted Investments,Money Market Investments,Federal Funds Sold & Securities Purchased Under Agreements to Resell,Security Borrowed,...,Preferred Stock Equity,Common Stock Equity,Net Tangible Assets,Invested Capital,Tangible Book Value,Total Debt,Share Issued,Ordinary Shares Number,Preferred Shares Number,Treasury Shares Number
Bank,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,...,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM
12/31/2023,"3,875,393,000","900,303,000","624,151,000","29,066,000","595,085,000",--,--,"276,152,000","276,152,000","200,436,000",...,"27,404,000","300,474,000","263,497,000","737,011,000","236,093,000","436,537,000","4,104,933.90","2,876,658.59","391,850","1,228,275.30"
12/31/2022,"3,665,743,000","882,826,000","567,234,000","27,697,000","539,537,000",--,--,"315,592,000","315,592,000","185,369,000",...,"27,404,000","264,928,000","231,473,000","604,820,000","204,069,000","339,892,000","4,104,933.90","2,934,257.80","391,850","1,170,676.09"
12/31/2021,"3,743,567,000","1,002,532,000","740,834,000","26,438,000","714,396,000",--,--,"261,698,000","261,698,000","206,071,000",...,"34,838,000","259,289,000","237,436,000","613,888,000","202,598,000","354,599,000","4,104,933.90","2,944,149.15","391,850","1,160,784.75"
12/31/2020,"3,384,757,000","823,893,000","527,609,000","24,874,000","502,735,000","32,227,000","32,227,000","296,284,000","296,284,000","160,635,000",...,"30,063,000","249,291,000","225,926,000","576,184,000","195,863,000","326,893,000","4,104,933.90","3,049,434.46","280,850","1,055,499.44"


In [67]:
ndf = pd.DataFrame()
for asset in ca:
    ndf.insert(ndf.shape[1], asset, tdf[f'{asset}'])
for l in cl:
    ndf.insert(ndf.shape[1], l, tdf[f"{l}"])


In [68]:
bank = 'JPM'
quick = [bank]
curr = [bank]
liab = [bank]
qr = [bank]
cr = [bank]
for i in range(2, 6):
    
    q, a, l = get_qancl_yr(i, bank, quick_assets, current_assets, current_liabilities)
    # print(q, a, l)
    quick.append(q)
    curr.append(a)
    liab.append(l)
    qr.append(q/l)
    cr.append(a/l)
ndf.insert(ndf.shape[1], 'quickAssets', quick)
ndf.insert(ndf.shape[1], 'currentAssets', curr)
ndf.insert(ndf.shape[1], 'currentLiabilities', liab)
ndf.insert(ndf.shape[1], 'quickRaito', qr)
ndf.insert(ndf.shape[1], 'currentRatio', cr)
ndf.head()


,"Cash, Cash Equivalents & Federal Funds Sold",Trading Securities,Other Short Term Investments,Loans Held for Sale,Accounts receivable,Other Assets,Consumer Loan,Held To Maturity Securities,Total Deposits,Federal Funds Purchased And Securities Sold Under Agreement To Repurchase,Payables And Accrued Expenses,Current Debt And Capital Lease Obligation,Trading Liabilities,quickAssets,currentAssets,currentLiabilities,quickRaito,currentRatio
Bank,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM,JPM
12/31/2023,"900,303,000","411,613,000","192,485,000","3,985,000","107,363,000","297,521,000","294,320,000","369,848,000","2,400,688,000","216,535,000","317,954,000","44,712,000","139,581,000",1419279000000,2577438000000,3119470000000,0.454974,0.826242
12/31/2022,"882,826,000","360,112,000","196,699,000","3,970,000","125,189,000","285,729,000","258,371,000","425,305,000","2,340,179,000","202,613,000","339,982,000","44,027,000","126,835,000",1368127000000,2538201000000,3053636000000,0.448032,0.831206
12/31/2021,"1,002,532,000","330,865,000","290,257,000","8,688,000","102,570,000","302,476,000","251,520,000","363,707,000","2,462,303,000","194,340,000","303,071,000","53,594,000","114,577,000",1435967000000,2652615000000,3127885000000,0.459086,0.848054
12/31/2020,"823,893,000","372,481,000","355,951,000","7,873,000","90,503,000","314,411,000","235,404,000","201,821,000","2,144,257,000","215,209,000","294,208,000","45,208,000","99,558,000",1286877000000,2402337000000,2798440000000,0.459855,0.858456


#### Other Ratios

In [17]:
# get from data directly and analyse

## Compliance

### CAR - Capital Adequacy
*Under Basel III, all banks are required to have a Capital Adequacy Ratio of at least 8%*

In [18]:
basic = pd.read_csv("../../data/interim/US_Banks_Basic.csv")

In [19]:
t1 = '"Tier 1 (core) risk-based capital"'
t2 = '"Tier 2 risk-based capital"'
rwa = '"Total risk weighted assets"'

In [20]:
# for all banks
def get_car():
    queries = [t1, t2, rwa]
    banks = basic.bank.unique()
    car = pd.DataFrame()
    for query in queries:
        car_temp = basic.query(f"tag == {query}")
        car = pd.concat([car, car_temp], ignore_index=True)
    # for bank in banks:# calc car
    bank_cars = {}
    for idx, row in car.iterrows():
        b = row['bank']
        if b not in bank_cars:
            bank_cars[b] = row.iloc[2:]
        else:
            if row['tag'] == "Tier 2 risk-based capital":
                tier2 = row.iloc[2:]
                num = bank_cars[b]
                bank_cars[b] = [num[i] + tier2[i] for i in range(len(tier2))]
            if row['tag'] == 'Total risk weighted assets':
                weighted = row.iloc[2:]
                num = bank_cars[b]
                bank_cars[b] = [num[i] / weighted[i] for i in range(len(tier2))]
                
    return car, bank_cars
car_df, bank_cars = get_car()


In [21]:
(bank_cars['jpmorgan-chase-bank'][:6])

[0.17345533056157378,
 0.18453470338218325,
 0.18175139303629784,
 0.1844286266329814,
 0.1806011052693971,
 0.17477219653253404]

In [22]:
def car_comp(basic, bank, qtr):
    car_df, bank_cars = get_car()
    cars = bank_cars[bank]

    qtrs = basic.columns[2:].to_list()
    qtr_idx = qtrs.index(f'{qtr}')

    if cars[qtr_idx]*100 > 8:
        print(f"Compliant: {cars[qtr_idx]*100}%")
    else:
        print(f"Not Compliant: {cars[qtr_idx]*100}%")
        

bank = 'jpmorgan-chase-bank'
qtr = "2002-12-31"
car_comp(basic, bank, qtr)

Compliant: 11.127689518782852%


### But, Is the Bank Large Enough?
*Under the Federal Reserve Board's capital framework for bank holding companies, covered savings and loan holding companies, and U.S. intermediate holding companies with $100 billion or more in total consolidated assets, capital requirements are in part determined by the supervisory stress test results.*

In [23]:
def check_large(bank):
    totalAssets = basic.query(f"tag == 'Total Assets:'")
    isLarge = {}
    for idx, row in totalAssets.iterrows():
        b = row['bank']
        assets = row.iloc[2:]
        bools = []
        for asset in assets:
            if asset > 100*10**9:
                bools.append('L')
            else:
                bools.append('NL')
        isLarge[b] = bools
    return isLarge[bank]
isLarge = check_large('jpmorgan-chase-bank')


### The Capital Ratios 

*Under the guidelines, banking organizations are required to maintain a minimum Total risk-based capital ratio (total capital to risk-weighted assets) of 8% and a Tier 1 risk-based capital ratio of 4%.*

In [33]:
t1cr = '"Tier 1 risk-based capital ratio"'
t1lever = '"Core capital (leverage) ratio"'
totCapR = '"Total risk-based capital ratio"'
def get_capr():
    queries = [t1cr, t1lever, totCapR]
    ratios = pd.DataFrame()
    for query in queries:
        r_temp = basic.query(f"tag == {query}")
        ratios = pd.concat([ratios,r_temp], ignore_index=True)
rats = get_capr()
rats.head()

,tag,bank,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,...,2002-12-31,2001-12-31,2000-12-31,1999-12-31,1998-12-31,1997-12-31,1996-12-31,1995-12-31,1994-12-31,1993-12-31
0,Tier 1 risk-based capital ratio,jpmorgan-chase-bank,8.546429e+10,9.543683e+10,9.614290e+10,9.454898e+10,8.148831e+10,9.175049e+10,8.876534e+10,9.138355e+10,...,2.605064e+10,2.229014e+10,2.216836e+10,1.890059e+10,1.380836e+10,1.134099e+10,1.012835e+10,7.670106e+09,7.443640e+09,6.186091e+09
1,Tier 1 risk-based capital ratio,bank-of-america,8.546429e+10,9.543683e+10,9.614290e+10,9.454898e+10,8.148831e+10,9.175049e+10,8.876534e+10,9.138355e+10,...,2.605064e+10,2.229014e+10,2.216836e+10,1.890059e+10,1.380836e+10,1.134099e+10,1.012835e+10,7.670106e+09,7.443640e+09,6.186091e+09
2,Tier 1 risk-based capital ratio,u-s-bank,8.546429e+10,9.543683e+10,9.614290e+10,9.454898e+10,8.148831e+10,9.175049e+10,8.876534e+10,9.138355e+10,...,2.605064e+10,2.229014e+10,2.216836e+10,1.890059e+10,1.380836e+10,1.134099e+10,1.012835e+10,7.670106e+09,7.443640e+09,6.186091e+09
3,Tier 1 risk-based capital ratio,goldman-sachs-bank-usa,8.546429e+10,9.543683e+10,9.614290e+10,9.454898e+10,8.148831e+10,9.175049e+10,8.876534e+10,9.138355e+10,...,2.605064e+10,2.229014e+10,2.216836e+10,1.890059e+10,1.380836e+10,1.134099e+10,1.012835e+10,7.670106e+09,7.443640e+09,6.186091e+09
4,Tier 1 risk-based capital ratio,hsbc-bank-usa,8.546429e+10,9.543683e+10,9.614290e+10,9.454898e+10,8.148831e+10,9.175049e+10,8.876534e+10,9.138355e+10,...,2.605064e+10,2.229014e+10,2.216836e+10,1.890059e+10,1.380836e+10,1.134099e+10,1.012835e+10,7.670106e+09,7.443640e+09,6.186091e+09


In [34]:


def isCompliant(ratio_df, bank):
    isLarge = check_large(bank)
    bank_r = ratio_df.query(f"bank == '{bank}'")
    t1crcomp = []
    t1lcomp = []
    totcomp = []
    for idx, row in bank_r.iterrows():
        if row["tag"] == 'Tier 1 risk-based capital ratio':
            t1r = row.iloc[2:]
            for i in range(len(t1r)):
                if isLarge[i] == 'L':
                    t1r[i] = t1r[i]/10**10
                    if t1r[i] > 6: # basel 3 norms
                        t1crcomp.append((t1r[i],True))
                    else:
                        t1crcomp.append((t1r[i], False))
                else:
                    t1crcomp.append('notLarge')
    
        if row["tag"] == "Core capital (leverage) ratio": # > 4%
            t1r = row.iloc[2:]
            
            for i in range(len(t1r)):
                if isLarge[i] == 'L':
                    if t1r[i] > 5:
                        t1lcomp.append((t1r[i],True))
                    else:
                        t1lcomp.append((t1r[i],False))
                else:
                    t1lcomp.append('notLarge')
                
        if row["tag"] == "Total risk-based capital ratio": 
            t1r = row.iloc[2:]
            for i in range(len(t1r)):
                if isLarge[i] == 'L':
                    if t1r[i] > 8:
                        totcomp.append((t1r[i],True))
                    else:
                        totcomp.append((t1r[i],False))
                else:
                    totcomp.append('notLarge')
        
    return t1crcomp, totcomp, t1lcomp


In [37]:
def get_comp(ratio_df, bank, qtr):
    t1, tot, t1l = isCompliant(ratio_df, f'{bank}')
    qtrs = ratio_df.columns[2:].to_list()
    qtr_idx = qtrs.index(f'{qtr}')
    print( f"Tier 1 Capital Ratio(RB): {t1[qtr_idx][0], t1[qtr_idx][1]}, Total Capital Ratio(RB): {tot[qtr_idx][0], tot[qtr_idx][1]}, Tier 1 Leverage Ratio: {t1l[qtr_idx][0], t1l[qtr_idx][1]}")


ratio_df = get_capr()
qtr = "2023-12-31"
bank = "the-bank-of-new-york-mellon"
get_comp(ratio_df, bank, qtr)


Tier 1 Capital Ratio(RB): (8.546428871054939, True), Total Capital Ratio(RB): (16.4074, True), Tier 1 Leverage Ratio: (6.6516, True)


In [127]:
banks = basic.bank.unique()
banks

array(['jpmorgan-chase-bank', 'bank-of-america', 'u-s-bank',
       'goldman-sachs-bank-usa', 'hsbc-bank-usa', 'morgan-stanley-bank',
       'mitsubishi-ufj-trust-banking-corporation-u-s-a', 'pnc-bank',
       'charles-schwab-bank-ssb',
       'sumitomo-mitsui-trust-bank-u-s-a-limited', 'truist-bank',
       'wells-fargo-bank', 'td-bank', 'the-huntington-national-bank',
       'citizens-bank-57957', 'citibank', 'fifth-third-bank',
       'regions-bank', 'comerica-bank', 'american-express-national-bank',
       'capital-one', 'the-bank-of-new-york-mellon'], dtype=object)

## Extract Ratios

In [84]:
basic = pd.read_csv("../../data/raw/uslocs/banks_basic_final.csv")
basic.head(2)

,Tag,Bank,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30,2022-03-31,...,2002-12-31,2001-12-31,2000-12-31,1999-12-31,1998-12-31,1997-12-31,1996-12-31,1995-12-31,1994-12-31,1993-12-31
0,# of Employees:,jpmorgan-chase-bank,"226,822","226,649","219,830","221,852","220,271","216,524","210,348","207,949",...,"77,286","67,401","56,567","41,563","42,229","41,670","41,615","24,042","25,546","26,409"
1,Total Assets:,jpmorgan-chase-bank,"$3,395,126,000,000","$3,385,581,000,000","$3,382,195,000,000","$3,267,963,000,000","$3,201,942,000,000","$3,308,575,000,000","$3,380,824,000,000","$3,476,711,000,000",...,"$621,962,000,000","$537,826,000,000","$377,116,000,000","$332,198,000,000","$296,717,000,000","$297,061,000,000","$272,429,000,000","$147,120,000,000","$135,742,000,000","$115,510,000,000"


In [85]:
def date_to_qtr(date):
    yr = date[:4]
    q = date[5:7]
    if q == '12':
        qtr = 'Q4'
    if q == '09':
        qtr = 'Q3'
    if q == '06':
        qtr = 'Q2'
    if q == '03':
        qtr = 'Q1'    
    return (yr + " " + qtr)

In [86]:
cols = basic.columns[2:]
d2q = {col : date_to_qtr(col) for col in cols}
basic = basic.rename(columns=d2q)

In [87]:
basic = basic.fillna('0')

In [88]:
basic.head(2)

,Tag,Bank,2023 Q4,2023 Q3,2023 Q2,2023 Q1,2022 Q4,2022 Q3,2022 Q2,2022 Q1,...,2002 Q4,2001 Q4,2000 Q4,1999 Q4,1998 Q4,1997 Q4,1996 Q4,1995 Q4,1994 Q4,1993 Q4
0,# of Employees:,jpmorgan-chase-bank,"226,822","226,649","219,830","221,852","220,271","216,524","210,348","207,949",...,"77,286","67,401","56,567","41,563","42,229","41,670","41,615","24,042","25,546","26,409"
1,Total Assets:,jpmorgan-chase-bank,"$3,395,126,000,000","$3,385,581,000,000","$3,382,195,000,000","$3,267,963,000,000","$3,201,942,000,000","$3,308,575,000,000","$3,380,824,000,000","$3,476,711,000,000",...,"$621,962,000,000","$537,826,000,000","$377,116,000,000","$332,198,000,000","$296,717,000,000","$297,061,000,000","$272,429,000,000","$147,120,000,000","$135,742,000,000","$115,510,000,000"


In [89]:
percentages = pd.DataFrame(columns=basic.columns) # for 1 bank, also appending rows cos its small
bank = 'jpmorgan-chase-bank'
for idx, row in basic.iterrows():
    
    if row['Bank'] == bank:
        val = row['2023 Q4']
        if val.find('%') != -1:
            percentages.loc[percentages.shape[0]] = row.to_list()
            # percentages = pd.concat([percentages, n_df], ignore_index=True, axis=0)
percentages.head()

,Tag,Bank,2023 Q4,2023 Q3,2023 Q2,2023 Q1,2022 Q4,2022 Q3,2022 Q2,2022 Q1,...,2002 Q4,2001 Q4,2000 Q4,1999 Q4,1998 Q4,1997 Q4,1996 Q4,1995 Q4,1994 Q4,1993 Q4
0,% insured,jpmorgan-chase-bank,39.75%,40.86%,41.00%,39.20%,39.39%,38.91%,38.10%,37.08%,...,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
1,Percentage of unprofitable institutions,jpmorgan-chase-bank,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Yield on earning assets,jpmorgan-chase-bank,5.1189%,4.9590%,4.7678%,4.5497%,2.8446%,2.4275%,2.0780%,1.8937%,...,0,0,0,0,0,0,0,0,0,0
3,Cost of funding earning assets,jpmorgan-chase-bank,1.9710%,1.8481%,1.6997%,1.5120%,0.5278%,0.3192%,0.1539%,0.0796%,...,0,0,0,0,0,0,0,0,0,0
4,Net interest margin,jpmorgan-chase-bank,3.1479%,3.1109%,3.0680%,3.0377%,2.3167%,2.1083%,1.9241%,1.8141%,...,0,0,0,0,0,0,0,0,0,0


In [94]:
p = percentages.transpose()
p.columns = p.iloc[0]
p = p.iloc[1:, :]
p.columns
dump = ['Percentage of unprofitable institutions', 'Assets per employee ($millions)', 'Earnings coverage of net charge-offs (x)',
        'Efficiency ratio', ]
p = p.drop(dump, axis=1)
p.shape

(95, 53)

In [99]:
tdf.head()

Breakdown,Total Assets,"Cash, Cash Equivalents & Federal Funds Sold",Cash And Cash Equivalents,Cash,Cash And Due from Banks,Interest Bearing Deposits Assets,Money Market Investments,Securities and Investments,Trading Securities,Available for Sale Securities,...,Federal Funds Purchased And Securities Sold Under Agreement To Repurchase,Payables And Accrued Expenses,Current Accrued Expenses,Current Debt And Capital Lease Obligation,Current Debt,Other Current Borrowings,Trading Liabilities,Derivative Product Liabilities,Long Term Debt And Capital Lease Obligation,Long Term Debt
Bank,BAC,BAC,BAC,BAC,BAC,BAC,BAC,BAC,BAC,BAC,...,BAC,BAC,BAC,BAC,BAC,BAC,BAC,BAC,BAC,BAC
12/31/2023,"3,180,151,000","622,043,000","341,419,000","27,892,000","305,181,000","8,346,000","280,624,000","1,017,946,000","146,539,000","10,203,000",...,"283,887,000",-,-,"32,098,000","32,098,000","32,098,000","95,530,000","43,432,000","302,204,000","302,204,000"
12/31/2022,"2,819,627,000","237,462,000","36,430,000","7,259,000","304,058,000","180,603,000","12,110,000","220,788,000","47,179,000","1,038,974,000",...,"24,510,000","207,003,000","-1,656,000","28,397,000","248,414,000","547,714,000","179,463,000","65,452,000","8,650,814","406,712"
12/31/2021,"3,169,495,000","606,085,000","355,365,000","29,222,000","318,999,000","7,144,000","250,720,000","1,126,273,000","143,646,000","8,895,000",...,"192,329,000","198,963,000","198,963,000","23,753,000","23,753,000","23,753,000","100,690,000","37,675,000","280,117,000","280,117,000"
12/31/2020,"505,036,000","387,009,000","199,869,000","6,546,000","1,043,422,000","107,344,000","632,825,000","234,491,000","1,026,344,000","920,006,000",...,"58,953,000","164,088,000","549,179,000","24,510,000","204,175,000","530,669,000","302,914,000",-,"7,996,778","250,321"


In [69]:
ndf.to_csv("../../outputs/liquidity.csv")

In [98]:
p.to_csv("../../outputs/ratios.csv")